In [65]:
import logging
import os
import re
from kiteconnect import KiteConnect
from dotenv import load_dotenv
load_dotenv()
import bs_threading
import importlib
import time
from datetime import datetime, timedelta
import dateutil
import pandas as pd
importlib.reload(bs_threading)
from bs_threading import bs_threadify, bs_make_throttle_ready_func


In [18]:
# load all the stocks
# we do it together to the final of the sytem skk

INPUT_KITE_HISTORICAL_DIR = "input/kite_historical"
INPUT_KITE_INSTRUMENTS_DIR = "input/kite_instruments"

In [3]:
df_all_instruments = pd.read_csv(os.path.join(INPUT_KITE_INSTRUMENTS_DIR,"instrument_list.csv"))
df_all_instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83505,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83506,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83507,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83508,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE


In [14]:
df_nifty = df_all_instruments[df_all_instruments["segment"].str.lower().str.startswith("indices") & df_all_instruments["name"].str.lower().str.startswith("nifty")]
df_nifty

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
15211,260617,1018,NIFTY 100,NIFTY 100,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15212,264457,1033,NIFTY 200,NIFTY 200,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15213,256265,1001,NIFTY 50,NIFTY 50,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15214,268041,1047,NIFTY 500,NIFTY 500,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15215,265993,1039,NIFTY ALPHA 50,NIFTY ALPHA 50,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15216,273673,1069,NIFTY ALPHALOWVOL,NIFTY ALPHALOWVOL,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15217,263433,1029,NIFTY AUTO,NIFTY AUTO,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15218,260105,1016,NIFTY BANK,NIFTY BANK,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15219,257289,1005,NIFTY COMMODITIES,NIFTY COMMODITIES,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15220,257545,1006,NIFTY CONSUMPTION,NIFTY CONSUMPTION,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE


In [5]:
for directory, _, filenames in os.walk(INPUT_KITE_HISTORICAL_DIR):
    for filename in filenames:
        print(filename)

1000449_PREMIERPOL_EQ_NSE_NSE_15minute.csv
1000449_PREMIERPOL_EQ_NSE_NSE_day.csv
1003009_NATCOPHARM_EQ_NSE_NSE_15minute.csv
1003009_NATCOPHARM_EQ_NSE_NSE_day.csv
1008129_INDLMETER_EQ_NSE_NSE_15minute.csv
1008129_INDLMETER_EQ_NSE_NSE_day.csv
1009665_GS261023C-GS_EQ_NSE_NSE_day.csv
1009921_EASUNREYRL-BZ_EQ_NSE_NSE_15minute.csv
1009921_EASUNREYRL-BZ_EQ_NSE_NSE_day.csv
1010177_GS101123C-GS_EQ_NSE_NSE_day.csv
1010945_JAKHARIA-SM_EQ_NSE_NSE_15minute.csv
1010945_JAKHARIA-SM_EQ_NSE_NSE_day.csv
101121_BEML_EQ_NSE_NSE_15minute.csv
101121_BEML_EQ_NSE_NSE_day.csv
1012481_GANGAFORGE-SM_EQ_NSE_NSE_15minute.csv
1012481_GANGAFORGE-SM_EQ_NSE_NSE_day.csv
1014529_GOCLCORP_EQ_NSE_NSE_15minute.csv
1014529_GOCLCORP_EQ_NSE_NSE_day.csv
1015041_REMSONSIND_EQ_NSE_NSE_15minute.csv
1015041_REMSONSIND_EQ_NSE_NSE_day.csv
1018881_TNPL_EQ_NSE_NSE_15minute.csv
1018881_TNPL_EQ_NSE_NSE_day.csv
101889_SGBNOV25VI-GB_EQ_NSE_NSE_15minute.csv
101889_SGBNOV25VI-GB_EQ_NSE_NSE_day.csv
1020673_GREENPLY_EQ_NSE_NSE_15minute.csv
10

5620993_INDTERRAIN_EQ_NSE_NSE_day.csv
563201_SGBFEB29XI-GB_EQ_NSE_NSE_15minute.csv
563201_SGBFEB29XI-GB_EQ_NSE_NSE_day.csv
56321_IEX_EQ_NSE_NSE_15minute.csv
56321_IEX_EQ_NSE_NSE_day.csv
5633_ACC_EQ_NSE_NSE_15minute.csv
5633_ACC_EQ_NSE_NSE_day.csv
563713_MIRCELECTR_EQ_NSE_NSE_15minute.csv
563713_MIRCELECTR_EQ_NSE_NSE_day.csv
563969_BIRET-RR_EQ_NSE_NSE_15minute.csv
563969_BIRET-RR_EQ_NSE_NSE_day.csv
567041_ABFRLPP1-E1_EQ_NSE_NSE_15minute.csv
567041_ABFRLPP1-E1_EQ_NSE_NSE_day.csv
568065_ERFLNCDI-N3_EQ_NSE_NSE_15minute.csv
568065_ERFLNCDI-N3_EQ_NSE_NSE_day.csv
5691649_DHUNINV_EQ_NSE_NSE_15minute.csv
5691649_DHUNINV_EQ_NSE_NSE_day.csv
5693185_EBBETF0431_EQ_NSE_NSE_15minute.csv
5693185_EBBETF0431_EQ_NSE_NSE_day.csv
5693953_SGBJUL28IV-GB_EQ_NSE_NSE_15minute.csv
5693953_SGBJUL28IV-GB_EQ_NSE_NSE_day.csv
5694209_EBBETF0425_EQ_NSE_NSE_15minute.csv
5694209_EBBETF0425_EQ_NSE_NSE_day.csv
5697281_NDRAUTO_EQ_NSE_NSE_15minute.csv
5697281_NDRAUTO_EQ_NSE_NSE_day.csv
5702913_WORTH_EQ_NSE_NSE_15minute.csv


In [91]:
def instrument_to_file(instrument_dict, interval="day"):
    kite_instrument_token = instrument_dict['instrument_token']
    trading_symbol = instrument_dict['tradingsymbol']
    instrument_type = instrument_dict['instrument_type']
    segment = instrument_dict['segment']
    exchange = instrument_dict['exchange']
    # we've tested that _ is valid seperator by evaluating for all the instruments (86k)
    result_file = f"{kite_instrument_token}_{trading_symbol}_{instrument_type}_{segment}_{exchange}_{interval}.csv"
    return result_file




In [92]:
for idx, row in df_nifty.iterrows():
    instrument_file_name = instrument_to_file(row, interval="5minute")
    if os.path.exists(os.path.join(INPUT_KITE_HISTORICAL_DIR,instrument_file_name)):
        print(INPUT_KITE_HISTORICAL_DIR+"/"+instrument_file_name)

input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_5minute.csv
input/kite_historical/268041_NIFTY 500_EQ_INDICES_NSE_5minute.csv


In [93]:
df_nifty_50_5min = pd.read_csv("input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_5minute.csv")
df_nifty_50_5min
df_nifty_50_day = pd.read_csv("input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_day.csv")
df_nifty_50_day

,date,open,high,low,close,volume,oi
0,1996-01-01 00:00:00+05:30,913.11,913.11,906.15,908.01,0,0.0
1,1996-01-02 00:00:00+05:30,906.78,906.78,899.52,900.38,0,0.0
2,1996-01-03 00:00:00+05:30,901.66,904.48,895.48,895.96,0,0.0
3,1996-01-04 00:00:00+05:30,891.37,891.37,884.10,889.67,0,0.0
4,1996-01-05 00:00:00+05:30,887.79,887.79,881.74,884.73,0,0.0
...,...,...,...,...,...,...,...
6320,2021-06-18 00:00:00+05:30,15756.50,15761.50,15450.90,15683.35,0,0.0
6321,2021-06-21 00:00:00+05:30,15525.85,15765.15,15505.65,15746.50,0,0.0
6322,2021-06-22 00:00:00+05:30,15840.50,15895.75,15752.10,15772.75,0,0.0
6323,2021-06-23 00:00:00+05:30,15862.80,15862.95,15673.95,15686.95,0,0.0


numpy.float64

In [171]:
class INDICATORS:
    MOVING_AVERAGE = "moving_average"
    EXPONENTIAL_MOVING_AVERAGE = "exponential_moving_average"
    
    @staticmethod
    def resolve_indicator_name(indicator_name):
        # valid: moving_average, moving average, ma, MoVingAveRage
        indicator_name_fixed = re.sub("[^a-zA-Z]+", "", indicator_name)
        if indicator_name_fixed.lower() in ['movingaverage', 'ma']:
            return INDICATORS.MOVING_AVERAGE
        return indicator_name
    
    @staticmethod
    def get_moving_average_indicator(df, length=5, source='close'):
        return df[source].rolling(length).mean()
    
    @staticmethod
    def create_get_indicator_func(df):
        def get_indicator_func(indicator_name, indicator_params_dict=None):
            indicator_name = INDICATORS.resolve_indicator_name(indicator_name)
            if indicator_name in df.columns:
                return df[indicator_name]

            if indicator_name == INDICATORS.MOVING_AVERAGE:
                if indicator_params_dict == None:
                    indicator_params_dict = {}
                if 'length' not in indicator_params_dict:
                    indicator_params_dict['length'] = 5
                if 'source' not in indicator_params_dict:
                    indicator_params_dict['source'] = 'close'
                
                df_column_selector = f"{indicator_name}_{indicator_params_dict['length']}_{indicator_params_dict['source']}"
                
                if df_column_selector in df.columns:
                    return df[df_column_selector]
                else:
                    source_column = indicator_params_dict['source']
                    df[df_column_selector] = INDICATORS.get_moving_average_indicator(df, length=indicator_params_dict['length'], source=source_column)
                    return df[df_column_selector]
                
        return get_indicator_func
            
        
indicator_func = INDICATORS.create_get_indicator_func(df_nifty_50_day)
indicator_func('moving average', {'length':25})
# df_nifty_50_day

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
6320    15488.236
6321    15521.170
6322    15547.756
6323    15574.028
6324    15609.404
Name: moving_average_25_close, Length: 6325, dtype: float64

In [105]:
ma25 = indicator_func('moving average', {'length':25})

25     868.1952
26     869.4608
27     872.8332
28     878.5672
29     883.9184
30     890.2244
31     897.5392
32     904.8368
33     910.8292
34     916.6784
35     921.8308
36     928.0864
37     934.9552
38     942.3912
39     949.7296
40     957.0528
41     963.4372
42     970.6684
43     978.0252
44     984.9064
45     990.9816
46     996.5896
47    1000.8184
48    1003.1348
49    1005.0972
Name: moving_average_25_close, dtype: float64

In [185]:
class BackTestOrder():
    
    def __init__(self, balance):
        self.balance = balance
        self.transactions = []
        self.symbol_stock = {}
    
    def get_balance(self, ):
        return self.balance
    
    def get_transactions(self, symbol):
        print(type(symbol), type(self.transactions[0]['symbol']))
        return [x for x in self.transactions if x['symbol']==symbol]
    
    def get_symbol_qty(self, symbol):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
        return self.symbol_stock[symbol]
    
    def buy(self, symbol, date, price, qty = None, amount = None):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
        
        if qty == -1:
            amount = self.balance
        
        if qty is not None and qty != -1:
            amount = price * qty
            
        elif amount is not None:
            qty = amount/price
            
        self.balance -= amount
        
        if qty > 0:
            self.symbol_stock[symbol] += qty
            self.transactions.append({"symbol": symbol, "date": date, "price":price, "qty":qty, "amount":amount})
        
    def sell(self, symbol, date, price, qty = None, amount = None):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
            
        if qty == -1:
            qty = self.symbol_stock[symbol]
            
        if qty is not None and qty != -1:
            amount = price * qty
        elif amount is not None:
            qty = amount/price
        self.balance += amount
        
        if qty > 0:
            self.symbol_stock[symbol] -= qty
            self.transactions.append({"symbol": symbol, "date": date, "price":price, "qty":qty, "amount":amount})
            

def _run_backtest(df, order_func):
    all_indicators = {}
    current_idx = 0
    indicator_func = INDICATORS.create_get_indicator_func(df)
    backtest_order_api = BackTestOrder(100000)
    def _get_indicator(indicator_name, indicator_params_dict=None):
        nonlocal all_indicators
        nonlocal indicator_func
        indicator_ukey = f"{indicator_name}{indicator_params_dict}"
        if indicator_ukey not in all_indicators:
            all_indicators[indicator_ukey] = indicator_func(indicator_name, indicator_params_dict)
        return all_indicators[indicator_ukey].iloc[:current_idx+1]
    for idx, row in df.iterrows():
#         print(idx)
        current_idx = idx
        order_func(_get_indicator, backtest_order_api)
    for transaction in backtest_order_api.get_transactions(df['symbol'].iloc[-1]):
        print(transaction)
        

df_nifty = df_all_instruments[df_all_instruments["segment"].str.lower().str.startswith("indices") &
                                  df_all_instruments["name"].str.lower().str.startswith("nifty 50")]
interval = "day"

# fetch all the instruments file names
instrument_files_for_backtest = []
for idx, row in df_nifty.iterrows():
    instrument_file_name = instrument_to_file(row, interval=interval)
    if os.path.exists(os.path.join(INPUT_KITE_HISTORICAL_DIR,instrument_file_name)):
#         print(INPUT_KITE_HISTORICAL_DIR+"/"+instrument_file_name)
        instrument_files_for_backtest.append(os.path.join(INPUT_KITE_HISTORICAL_DIR, instrument_file_name))

for instrument_file in instrument_files_for_backtest:
    df = pd.read_csv(instrument_file)
    df['symbol'] = os.path.split(instrument_file)[-1].split(".")[0]
    
    print("------------------")
    print(instrument_file)
    print("------------------")
    
    def order_func(get_indicator, backtest_order_api):
        close = get_indicator("close")
        symbol = get_indicator("symbol")
        date = get_indicator("date")
        ma15 = get_indicator("moving average", {'length':15})
        ma100 = get_indicator("moving average", {'length':100})
        if len(ma15)<2 or ma100.iloc[-2] == None:
            return
#         print(ma15.iloc[-2], ma100.iloc[-2] ,"->", ma15.iloc[-1], ma100.iloc[-1])
        if ma15.iloc[-2] <= ma100.iloc[-2] and ma15.iloc[-1] > ma100.iloc[-1]:
#             print("--------- buy ----------")
            backtest_order_api.buy(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
        if ma15.iloc[-2] >= ma100.iloc[-2] and ma15.iloc[-1] < ma100.iloc[-1]:
#             print("--------- sell ----------")
            backtest_order_api.sell(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
#         print(symbol.iloc[-1], ma.iloc[-1])
#         print(backtest_order_api.get_symbol_qty(symbol.iloc[-1]), backtest_order_api.get_balance())
        
    _run_backtest(df,order_func)

------------------
input/kite_historical\256265_NIFTY 50_EQ_INDICES_NSE_day.csv
------------------
<class 'str'> <class 'str'>
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1997-01-13 00:00:00+05:30', 'price': 980.35, 'qty': 102.0043861886061, 'amount': 100000.0}
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1997-09-16 00:00:00+05:30', 'price': 1141.0, 'qty': 102.0043861886061, 'amount': 116387.00464119957}
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1998-03-09 00:00:00+05:30', 'price': 1081.45, 'qty': 107.62125354033896, 'amount': 116387.00464119957}
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1998-06-08 00:00:00+05:30', 'price': 993.5, 'qty': 107.62125354033896, 'amount': 106921.71539232675}
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1998-12-30 00:00:00+05:30', 'price': 872.65, 'qty': 122.52531414923136, 'amount': 106921.71539232675}
{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'date': '1999-11-16 00:00:00+05:30',

<class 'str'> <class 'str'>
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2000-09-08 00:00:00+05:30', 'price': 1062.55, 'qty': 94.11321820149641, 'amount': 100000.0}
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2000-09-27 00:00:00+05:30', 'price': 929.25, 'qty': 94.11321820149641, 'amount': 87454.70801374054}
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2000-12-13 00:00:00+05:30', 'price': 987.55, 'qty': 88.5572457229918, 'amount': 87454.70801374054}
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2001-01-03 00:00:00+05:30', 'price': 922.0, 'qty': 88.5572457229918, 'amount': 81649.78055659843}
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2001-01-18 00:00:00+05:30', 'price': 928.3, 'qty': 87.95624319357798, 'amount': 81649.78055659843}
{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'date': '2001-03-12 00:00:00+05:30', 'price': 852.55, 'qty': 87.95624319357798, 'amount': 74987.0951346849}
{'symbol': '268041_NIFTY 500_EQ

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
15213,256265,1001,NIFTY 50,test,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15214,268041,1047,NIFTY 500,test,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
